In [12]:
import torch
import numpy as np
import pandas as pd
import glob
import os
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.io import read_image 
from torchvision import datasets, models, transforms
import math 
import random 
import cv2
from PIL import Image
import pickle
import imageio
import cv2
import kornia as K
import kornia.geometry as KG
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from numpy.linalg import inv
from numpy.matlib import repmat
import torchvision.models as models
from dataset import *

vgg = models.vgg16(pretrained=True)

base_dir = 'code/unspervised_vis_loc_reproduce/'


c:\Users\28340\anaconda3\envs\fyp\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\28340\anaconda3\envs\fyp\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
vgg.avgpool = torch.nn.Identity()
vgg.features[30] = torch.nn.Identity()
for param in vgg.features:
    param.requires_grad = False

In [14]:
class Upsampling(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=0):
        super(Upsampling, self).__init__()
        # print("in_channels: ",in_channels)
        # print("out_channels: ",out_channels)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.upsample = torch.nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
                                     
    def forward(self, x1,x2,final = False):
        x1up = self.upsample(x1)
        # print("x1up?: ",x1up.shape)
        # print("x2.shape: ",x2.shape)
        if x1up.shape != x2.shape:
            x1up = torch.nn.functional.pad(x1up, (0, x2.shape[3] - x1up.shape[3], 0, x2.shape[2] - x1up.shape[2]))

        x = torch.cat([x2, x1up], dim=1)
        if final:
            output= torch.nn.Sequential(torch.nn.Conv2d(64+17, 17, kernel_size= 2, padding= 0,stride=1),
                                     torch.nn.BatchNorm2d(17),
                                     torch.nn.ReLU(True),
                                     
                                     torch.nn.Conv2d(17,17, kernel_size=1, padding= 0,stride=1),
                                     torch.nn.Sigmoid())(x)
            return output
        output = torch.nn.Sequential(torch.nn.Conv2d(self.in_channels, self.out_channels, kernel_size= 2, padding= 0,stride=1),
                                     torch.nn.BatchNorm2d(self.out_channels),
                                     torch.nn.ReLU(True),
                                     torch.nn.Conv2d(self.out_channels, self.out_channels, kernel_size= 2, padding= 0,stride=1),
                                     torch.nn.BatchNorm2d(self.out_channels),
                                     torch.nn.ReLU(True),
                                     torch.nn.Conv2d(self.out_channels, self.out_channels, kernel_size= 2, padding= 0,stride=1),
                                     torch.nn.BatchNorm2d(self.out_channels),
                                     torch.nn.ReLU(True))(x)
        

        return output

In [15]:
class UNet(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        self.vgg = vgg

        self.up_1 = Upsampling(1024, 17)
        self.up_2 = Upsampling(256+17, 17)
        self.up_3 = Upsampling(128+17, 17)
        self.up_4 = Upsampling(64+17, 17)
        # self.up_4 =
        # self.outputs = LastConv(64+17, 17)
    
    def forward(self,x):
        x1 = self.vgg.features[0:4](x)
        x2 = self.vgg.features[4:9](x1)
        x3 = self.vgg.features[9:16](x2)
        x4 = self.vgg.features[16:23](x3)
        x5 = self.vgg.features[23:30](x4)


        x6 = self.up_1(x5,x4)
        x7 = self.up_2(x6,x3)
        x8 = self.up_3(x7,x2)
        x9 = self.up_4(x8,x1,True)

        # (F3,S3),(F2,S2),(F1,S1),(F0,S0)
        return (x6[:15],x6[15:]),(x7[:15],x7[15:]),(x8[:15],x8[16:]),(x9[:15],x9[16:])
        


In [16]:
featureMap = UNet(3,17)

In [17]:
dataset = Carla_Dataset(img_dir='carla_npzs')

In [18]:
(F3,S3),(F2,S2),(F1,S1),(F0,S0) = featureMap(dataset[0][0].unsqueeze(0))

AttributeError: 'numpy.ndarray' object has no attribute 'unsqueeze'